# Advent of Code - 2017

Inspired by [people-who-know-what-they-are-doing](https://nbviewer.jupyter.org/url/norvig.com/ipython/Advent%20of%20Code.ipynb), I decided to do the [Advent of Code](http://adventofcode.com/2017), and to do it in a Jupyter Notebook.

I'd say there's about a 50/50 chance that I finish this... (:

## [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha (part 1)

In [1]:
with open('./inputs/input_day_01.txt', 'r') as f:
    input1 = list(f)[0]

In [2]:
def digit_str_to_list(s):
    l = list(s.strip('\n'))
    l = [int(e) for e in l]    
    return l

def circ_sum_adj_reps(s):
    l = digit_str_to_list(s)
    adj_rep_sum = 0
    last = l[-1]
    for curr in l:
        if last == curr:
            adj_rep_sum += last
        last = curr
    return adj_rep_sum

In [3]:
# Test cases
assert (3 == circ_sum_adj_reps('1122'))
assert (4 == circ_sum_adj_reps('1111'))
assert (0 == circ_sum_adj_reps('1234'))
assert (9 == circ_sum_adj_reps('91212129'))

In [4]:
print(circ_sum_adj_reps(input1))

1216


## [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha (part 2)

In [5]:
def circ_sum_hlfwy_reps(s):
    l = digit_str_to_list(s)
    hlfwy_rep_sum = 0

    n = len(l)
    for i in range(n):
        curr_elt = l[i]
        hlfwy_adj_elt = l[ (i + n//2) % n]
        
        if curr_elt == hlfwy_adj_elt:
            hlfwy_rep_sum += curr_elt
            
    return hlfwy_rep_sum

In [6]:
# Test cases
assert( 6 == circ_sum_hlfwy_reps('1212'))
assert( 0 == circ_sum_hlfwy_reps('1221'))
assert( 0 == circ_sum_hlfwy_reps('1221'))
assert( 4 == circ_sum_hlfwy_reps('123425'))
assert( 12 == circ_sum_hlfwy_reps('123123'))
assert( 4 == circ_sum_hlfwy_reps('12131415'))

In [7]:
print(circ_sum_hlfwy_reps(input1))

1072


## [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum (part 1)

What is the checksum for the spreadsheet in your puzzle input?


In [8]:
with open('./inputs/input_day_02.txt', 'r') as f:
    input2 = list(f)
    input2 = [x.strip('\n').split('\t') for x in input2]
    input2 = [[int(e) for e in l] for l in input2]

In [9]:
def check_sheet(sheet):
    sheet_checksum = 0
    for row in sheet:
        sheet_checksum += (max(row) - min(row))
    return sheet_checksum

In [10]:
assert( 18 == check_sheet([[5,1,9,5], [7,5,3], [2,4,6,8]]))

In [11]:
print(check_sheet(input2))

53460


## [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum (part 2)

What is the sum of each row's result in your puzzle input?

In [12]:
def find_unique_divible_pair(l):
    n = len(l)
    ls = sorted(l)
    for i in range(n):
        for j in range(i+1,n):
            if (ls[j] % ls[i] == 0):
                break
        else:
            continue
        break
    return ls[i], ls[j]

# Test the divisible pair function
assert ( (2,8) == find_unique_divible_pair([5,9,2,8]))
assert ( (3,9) == find_unique_divible_pair([9,4,7,3]))
assert ( (3,6) == find_unique_divible_pair([3,8,6,5]))

In [13]:
def sheet_pair_sum(sheet):
    sheet_sum = 0
    for row in sheet:
        d1, d2 = find_unique_divible_pair(row)
        sheet_sum += (d2 // d1)
    return sheet_sum

# Test the sheet pair sum function
assert( 9 == sheet_pair_sum([[5,9,2,8], [9,4,7,3], [3,8,6,5]]))

In [14]:
print(sheet_pair_sum(input2))

282


## [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory

Your puzzle input is *361527.*

### Some quick thoughts
- First off, the problem could be thought of in terms of nested $ n \times n $ squares.

In [15]:
input3 = 361527

In [16]:
from numpy import sqrt, floor, ceil

def prev_odd_sqrt(n):
    if int(ceil(sqrt(n)))**2 == n:
        n = n - 1
    prev_sqrt = int(floor(sqrt(n)))

    if prev_sqrt % 2 == 0:
        prev_sqrt -= 1
    
    return prev_sqrt


def relative_position(n):
    if n == 1:
        x, y = 0,0 
    else:
        m = prev_odd_sqrt(n) 
        m_squared = m**2
        n_square = m // 2 + 1
        edge_len = 2 * n_square
        ordered_edge = int(ceil((n - m_squared) / (m + 1.0)))
        
        if ordered_edge == 1:
            x =   n_square
            y = (-n_square + 1) + (n - (m_squared + 1)) 
                        
        elif ordered_edge == 2:
            x =  n_square - (n - (m_squared + edge_len ))            
            y =  n_square
                        
        elif ordered_edge == 3:
            x = - n_square
            y = n_square - (n - (m_squared + 2 * edge_len))
        
        elif ordered_edge == 4:
            x = - n_square + (n - (m_squared + 3 * edge_len))
            y = - n_square
    
    return x, y

def manhattan_spiral_distance(n):
    row, col = relative_position(n)
    return abs(row) + abs(col)

In [17]:
assert (manhattan_spiral_distance(1) == 0)
assert (manhattan_spiral_distance(12) == 3)
assert (manhattan_spiral_distance(23) == 2)
assert (manhattan_spiral_distance(1024) == 31)

In [18]:
print (manhattan_spiral_distance(input3))

326


## [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory (part 2)

Your puzzle input is still *361527.*

### Quick thoughts
One way that we can do this is to figure out what cells are adjacent to any cell, and to figure out which already have been computed.

In [19]:
def edge_helper(n):
    m = prev_odd_sqrt(n) 
    m_squared = m**2
    n_square = m//2 + 1
    edge_len = 2 * n_square
    ordered_edge = int(ceil((n - m_squared) / (m + 1.0)))
    
    return m, m_squared, n_square, edge_len, ordered_edge

def get_first_on_edge(n_square, ordered_edge):
    m = 2 * (n_square - 1) + 1
    m_squared = m**2
    edge_len = 2 * n_square
    return m_squared + (ordered_edge - 1) * edge_len + 1

def get_last_on_edge(n_square, ordered_edge):
    m = 2 * (n_square - 1) + 1
    m_squared = m**2
    edge_len = 2 * n_square
    return m_squared + ordered_edge * edge_len

def is_first_on_edge(n):    
    if n == 1:
        first_on_edge = True
    else:
        m, m_squared, n_square, edge_len, ordered_edge = edge_helper(n)
        is_first_on_edge = (n == get_first_on_edge(n_square, ordered_edge))        
    return is_first_on_edge

def is_last_on_edge(n):    
    if n == 1:
        last_on_edge = True
    else:
        m, m_squared, n_square, edge_len, ordered_edge = edge_helper(n)
        is_last_on_edge = (n == get_last_on_edge(n_square, ordered_edge))
    return is_last_on_edge

def left(n):
    if n == 1:
        left = 6
    else:
        m, m_squared, n_square, edge_len, ordered_edge = edge_helper(n)
        if ordered_edge == 1:
            if is_first_on_edge(n):
                left = m_squared
            elif is_last_on_edge(n):
                left = n + 1
            else:
                left = get_first_on_edge(n_square-1, 1) + n - get_first_on_edge(n_square, 1) - 1

        elif ordered_edge == 2:
            if not is_last_on_edge(n):
                left = n + 1
            else:
                left = get_first_on_edge(n_square+1, 3)
                
        elif ordered_edge == 3:
            left = get_first_on_edge(n_square+1, 3) + (n - get_first_on_edge(n_square, 3)) + 1

        elif ordered_edge == 4:
            left = n - 1
            
    return left

def right(n):
    if n == 1:
        right = 2
    else:
        m, m_squared, n_square, edge_len, ordered_edge = edge_helper(n)
        if ordered_edge == 1:
            right = get_first_on_edge(n_square+1, 1) + (n - get_first_on_edge(n_square, 1)) + 1

        elif ordered_edge == 2:
            right = n - 1
                
        elif ordered_edge == 3:
            if not is_last_on_edge(n):
                right = get_last_on_edge(n_square-1, 2) + (n - get_first_on_edge(n_square, 3))
            else:
                right = n + 1

        elif ordered_edge == 4:
            right = n + 1
            
    return right

def up(n):
    if n == 1:
        up = 4
    else:
        m, m_squared, n_square, edge_len, ordered_edge = edge_helper(n)
        if ordered_edge == 1:
            if not is_last_on_edge(n):
                up = n + 1
            else:
                up = get_first_on_edge(n_square + 1, 2)
        elif ordered_edge == 2:
            up = get_first_on_edge(n_square + 1, 2) + (n - get_first_on_edge(n_square, 2)) + 1
        elif ordered_edge == 3:
            up = n -1
        elif ordered_edge == 4:
            up = get_last_on_edge(n_square - 1, 3) + (n - get_first_on_edge(n_square, 4))
            
    return up

def down(n):
    if n == 1:
        down = 8
    else:
        m, m_squared, n_square, edge_len, ordered_edge = edge_helper(n)
        if ordered_edge == 1:
            if is_first_on_edge(n):
                down = get_last_on_edge(n_square, 4)
            else:
                down = n -1
        elif ordered_edge == 2:
            if is_last_on_edge(n):
                down = n + 1
            else:
                down = get_last_on_edge(n_square - 1, 1) + (n - get_first_on_edge(n_square, 2))
        elif ordered_edge == 3:
            if is_last_on_edge(n):
                down = get_first_on_edge(n_square + 1, 4)
            else:
                down = n + 1
        elif ordered_edge == 4:
            down = get_first_on_edge(n_square + 1, 4) + (n - get_first_on_edge(n_square, 4)) + 1            
    return down

def up_left(n):
    up_left = left(up(n))
    return up_left

def down_left(n):
    down_left = left(down(n))
    return down_left

def up_right(n):
    up_right = right(up(n))
    return up_right

def down_right(n):
    down_right = right(down(n))
    return down_right

def adjacent(n):
    return [right(n), up_right(n), up(n), up_left(n), left(n), down_left(n), down(n), down_right(n)]

def adjacent_below(n):
    return [x for x in adjacent(n) if x < n]
    

In [20]:
# Pad with 0 to index at 1 because I just want to be done with this...
def find_first_bigger_adj_sum(input_in):
    adjacent_sum_list = [0,1]
    i = 2
    while adjacent_sum_list[i - 1] < input_in :
        adjacent_sum_list.append(sum([adjacent_sum_list[kk] for kk in adjacent_below(i)]))
        i += 1
    return adjacent_sum_list[i-1]


In [21]:
print (find_first_bigger_adj_sum(input3))

363010


## [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases (part 1)

The system's full passphrase list is available as your puzzle input. How many passphrases are valid?

In [22]:
with open('./inputs/input_day_04.txt', 'r') as f:
    input4 = list(f)
    input4 = [x.strip('\n') for x in input4]

In [23]:
def is_valid_passphrase(phrase):
    phrase = phrase.split(' ')
    return len(phrase) == len(set(phrase))

def count_valid_phrases(phrase_list, phrase_validator):
    count = 0
    for phrase in phrase_list:
        if phrase_validator(phrase):
            count += 1
    return count

In [24]:
assert(is_valid_passphrase('aa bb cc dd ee') == True)
assert(is_valid_passphrase('aa bb cc dd aa') == False)
assert(is_valid_passphrase('aa bb cc dd aaa') == True)

In [25]:
print(count_valid_phrases(input4, is_valid_passphrase))

325


## [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases (part 2)

Under this new system policy, how many passphrases are valid?

In [26]:
def is_valid_passphrase_v2(phrase):
    phrase = phrase.split(' ')
    phrase = [''.join(sorted(list(word))) for word in phrase]
    return len(phrase) == len(set(phrase))

In [27]:
assert(True == is_valid_passphrase_v2('abcde fghij'))
assert(False == is_valid_passphrase_v2('abcde xyz ecdab'))
assert(True == is_valid_passphrase_v2('a ab abc abd abf abj'))
assert(True == is_valid_passphrase_v2('iiii oiii ooii oooi oooo'))
assert(False == is_valid_passphrase_v2('oiii ioii iioi iiio'))

In [28]:
print (count_valid_phrases(input4, is_valid_passphrase_v2))

119


## [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike (part 1)

How many steps does it take to reach the exit?

In [29]:
with open('./inputs/input_day_05.txt','r') as f:
    input5 = list(f)
    input5 = [int(x.strip('\n')) for x in input5]

In [30]:
def maze_jump(input_instructions):
    instruction_set = input_instructions[:]
    n_jumps = 0
    last_pos = 0
    current_pos = 0
    while ((current_pos >= 0) and (current_pos < len(instruction_set))):
        n_jumps += 1
        last_pos = current_pos
        current_pos = current_pos + instruction_set[current_pos]
        instruction_set[last_pos] = instruction_set[last_pos] + 1
    return n_jumps        

In [31]:
assert(5 == maze_jump([0,3,0,1,-3]))

In [32]:
print (maze_jump(input5))

375042


## [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike (part 2)

How many steps does it now take to reach the exit?

In [33]:
def maze_jump_v2(input_instructions):
    instruction_set = input_instructions[:]
    n_jumps = 0
    last_pos = 0
    current_pos = 0
    while ((current_pos >= 0) and (current_pos < len(instruction_set))):
        n_jumps += 1
        last_pos = current_pos
        current_pos = current_pos + instruction_set[current_pos]
        if (instruction_set[last_pos] >= 3):
            instruction_set[last_pos] = instruction_set[last_pos] - 1
        else:
            instruction_set[last_pos] = instruction_set[last_pos] + 1
    return n_jumps        

In [34]:
assert(10 == maze_jump_v2([0,3,0,1,-3]))

In [35]:
%%time
print (maze_jump_v2(input5))

28707598
CPU times: user 9.59 s, sys: 3.94 ms, total: 9.6 s
Wall time: 9.6 s


## [Day 6](http://adventofcode.com/2017/day/6): Memory Reallocation

Given the initial block counts in your puzzle input, how many redistribution cycles must be completed before a configuration is produced that has been seen before?

### Part 2

How many cycles are in the infinite loop that arises from the configuration in your puzzle input?


In [36]:
with open('./inputs/input_day_06.txt','r') as f:
    input6 = [int(e) for e in f.read().split('\t')]

In [37]:
import operator

def realloc_until_repeat(input_state):
    n_banks = len(input_state)
    n_reallocations = 0
    observed_states = [input_state[:]]
    current_state = input_state[:]
    seen_before = False
    
    while(not seen_before):                
        # find the biggest block
        max_block = max(current_state)
        idx_max_block = current_state.index(max_block)
        
        # remove the blocks from the biggest bank and
        # redistribute the biggest block amongst the banks
        current_state[idx_max_block] = 0
        undistributed_blocks = max_block
        idx_current = idx_max_block
        
        while (undistributed_blocks > 0):
            # increment to next bank to deposit a block
            # cycle back to 0 if we're at the last bank
            idx_current = (idx_current + 1) % n_banks 

            # deposit 1 block in the current bank
            current_state[idx_current] = current_state[idx_current] + 1
            undistributed_blocks = undistributed_blocks - 1

        # just finished a reallocation, record and update list of observed states
        n_reallocations += 1
        seen_before = current_state in observed_states
        observed_states.append(current_state[:])

    # which state did the repetition occur at?
    repeated_state = current_state
    cycles_to_repeat = observed_states.index(repeated_state)
    repeated_cycle_length = n_reallocations - cycles_to_repeat
    
    return n_reallocations, cycles_to_repeat, repeated_cycle_length

In [38]:
# part 1
assert(5 == realloc_until_repeat([0,2,7,0])[0])
# part 2
assert(4 == realloc_until_repeat([0,2,7,0])[2])

In [39]:
%%time
n_reallocations, cycles_to_repeat, repeated_cycle_length = realloc_until_repeat(input6)

CPU times: user 329 ms, sys: 4.02 ms, total: 333 ms
Wall time: 335 ms


In [40]:
print ("Day 6 Part 1: n reallocations before repetition = {}".format(n_reallocations))
print ("Day 6 Part 2: length of repeated cycle = {}".format(repeated_cycle_length))

Day 6 Part 1: n reallocations before repetition = 4074
Day 6 Part 2: length of repeated cycle = 2793


## [Day 7](http://adventofcode.com/2017/day/7): Recursive Circus

Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

In [41]:
with open('./inputs/input_day_07.txt','r') as f:
    input7 = list(f)

In [42]:
def remove_parens(string):
    return string.replace('(','').replace(')','')

def parse_instruction(instruction):
    # tidy up the instruction, and break it into its components
    instruction = instruction.replace('\n','')
    instr_list = instruction.split('->')    
    
    # get the program name and weight
    program_info = instr_list[0].split( )
    program_name = program_info[0]
    program_weight = int(remove_parens(program_info[1]))

    # get any children
    if len(instr_list) == 2:
        children_names = instr_list[1].replace(' ','').split(',')
    else:
        children_names = []
    
    return (program_name, program_weight, children_names)

class weighted_node:
    def __init__(self, name, weight, children_names):
        self.name = name
        self.weight = weight
        self.parent = None
        self.parent_name = ''
        self.children_names = children_names
        self.children = []
        self.weight_below = 0
    
    def __str__(self):
        print_str =  'name = {}\n'.format(self.name)
        print_str = print_str + 'weight = {}\n'.format(self.weight)
        print_str = print_str + 'parent = {}\n'.format(self.parent_name)
        print_str = print_str + 'children = {}\n'.format(self.children_names)
        print_str = print_str + 'weight below = {}\n'.format(self.weight_below)
        return print_str
    
    def set_parent(self, parent):
        self.parent = parent
        self.parent_name = parent.name
        parent.children.append(self)

    def update_weight_below(self):
        parent = self.parent
        while parent != None:
            # add nodes weight to parent's weight below
            parent.weight_below = parent.weight_below + self.weight
            parent = parent.parent        
    
    def is_balanced(self):
        if len(self.children) > 0:
            weights_below = [child.weight + child.weight_below for child in self.children]
            is_balanced = len(set(weights_below)) == 1
        else:
            is_balanced = True
        return is_balanced

class tower:
    def __init__(self, instruction_list):
        tower_nodes = []
        tower_names = []
        
        for curr_instruction in instruction_list:        
            # extract info from current instruction and 
            # create the current node
            name, weight, children_names = parse_instruction(curr_instruction)        
            curr_node = weighted_node(name, weight, children_names)
            
            # check if the node had a parent
            for node in tower_nodes:
                if name in node.children_names:
                    curr_node.set_parent(node)
                    break
            else:
                parent = None
            
            # check if the children are on the list
            for child_name in children_names:
                try:
                    idx_child = tower_names.index(child_name) 
                    child_node = tower_nodes[idx_child]
                    child_node.set_parent(curr_node)
                except ValueError:
                    pass
        
            # append the current node to the tower
            tower_nodes.append(curr_node)
            tower_names.append(name)
        
        # save the nodes only
        self.tower_nodes = tower_nodes
        self.set_weight_below()

    def set_weight_below(self):
        curr_level_nodes = [node for node in self.tower_nodes if len(node.children_names) == 0]
        next_level_nodes = list(set([node.parent for node in curr_level_nodes if node.parent != None]))
 
        while len(curr_level_nodes) > 1:
            for node in curr_level_nodes:
                node.update_weight_below()            
            curr_level_nodes = next_level_nodes
            next_level_nodes = list(set([node.parent for node in curr_level_nodes if node.parent != None]))
                            
    def find_base(self):
        for node in self.tower_nodes:
            if node.parent == None:
                break
        return node        

In [43]:
# test on example
instructions_str = '''pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)'''
instruction_list = instructions_str.split('\n')

# build tower
tow = tower(instruction_list)

# find base
base_node = tow.find_base()
#print base_node.name

In [44]:
tow = tower(input7)
base_node = tow.find_base()
print (base_node.name)

qibuqqg


## [Day 7](http://adventofcode.com/2017/day/7): Part Two 

Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

In [45]:
def find_corrected_weight(tower):
    unbalanced_nodes = filter( lambda node: not node.is_balanced(), tower.tower_nodes)
    
    # find the one unbalanced node whose children whose weigth + weight below is the same
    # for all but one child (ie the node which has exactly two values in this set)
    for node in unbalanced_nodes:
        below_weights = [child.weight + child.weight_below for child in node.children]
        distinct_below_weights = list(set(below_weights))
        if len(distinct_below_weights) == 2:
            break

    # how much extra weight is the bad node holding?
    max_weight = max(below_weights)
    min_weight = min(below_weights)
    excess_weight = max_weight - min_weight
    
    # select which of the children is the bad node, and calculate what it should weigh
    idx_max_child = below_weights.index(max_weight)
    bad_child = node.children[idx_max_child]
    corrected_weight = bad_child.weight - excess_weight
    
    return corrected_weight

In [46]:
find_corrected_weight(tow)

-3132

## [Day 8](http://adventofcode.com/2017/day/8): I Heard You Like Registers 

### Part 1:

What is the largest value in any register after completing the instructions in your puzzle input?

### Part 2:
To be safe, the CPU also needs to know the highest value held in any register during this process so that it can decide how much memory to allocate to these operations. For example, in the above instructions, the highest value ever held was 10 (in register c after the third instruction was evaluated).

In [47]:
with open('./inputs/input_day_08.txt','r') as f:
    input8 = [x.strip('\n') for x in list(f)]

In [48]:
def parse_instruction(instruction_str):
    instruction_list = instruction_str.split(' ')
    
    # parse instruction for current register
    active_register = instruction_list[0]
    register_op = instruction_list[1]
    register_increment = int(instruction_list[2])
    
    # parse the comparison
    comparison_register = instruction_list[4]
    comparison_op = instruction_list[5]
    comparison_value = int(instruction_list[6])
    
    return (active_register, register_op, register_increment, 
            comparison_register, comparison_op, comparison_value)

class Registry:
    def __init__(self, instruction_list = None):
        self.registers = dict()
        self.max_register_val_seen = float('-inf')
      
    def perform_instruction(self, instruction_str):
        (active_register, register_op, register_increment, 
         comparison_register, comparison_op, comparison_value) = parse_instruction(instruction_str)
        
        if active_register not in self.registers:
            self.registers[active_register] = 0
        if comparison_register not in self.registers:
            self.registers[comparison_register] = 0
    
        # perform the comparison
        comparsion_result = (
            eval('{} {} {}'.format(self.registers[comparison_register], comparison_op, comparison_value))
        )
        
        # handle the comparison result
        if comparsion_result:
            if register_op == 'inc':
                self.registers[active_register] += register_increment
            elif register_op == 'dec':
                self.registers[active_register] -= register_increment

        # check to see if the current value is the biggest seen
        active_register_value = self.registers[active_register]
        if active_register_value > self.max_register_val_seen:
            self.max_register_val_seen = active_register_value
            
    def update_registers(self, instruction_list):
        max_register_seen = float('-inf')
    
        for instruction_str in instruction_list:
            self.perform_instruction(instruction_str)
            
    def max_register(self):
        return max(self.registers.values())

In [49]:
instruction_str = '''b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10'''
instruction_list = instruction_str.split('\n')

reg = Registry()
reg.update_registers(instruction_list)
assert(1 == reg.max_register())

In [50]:
reg = Registry()
reg.update_registers(input8)

# part 1's answer
max_register_at_end = reg.max_register()
print('Day 8 Part 1: max register at end = {}'.format(max_register_at_end))

# part 2's answer:
max_register_seen = reg.max_register_val_seen
print('Day 8 Part 2: max register seen = {}'.format(max_register_seen))
 

Day 8 Part 1: max register at end = 4902
Day 8 Part 2: max register seen = 7037


## [Day 9](http://adventofcode.com/2017/day/9): Stream Processing (part 1)

What is the total score for all groups in your input?

In [51]:
with open('./inputs/input_day_09.txt','r') as f: 
    input9 = [x.strip('\n') for x in list(f)]
input9 = input9[0]

In [52]:
def score_garbage_stream(g_stream):
    in_group = False 
    group_depth = 0
    total_score = 0
    group_score = 0

    in_garbage = False
    ignore_c = False
    
    for c in g_stream:
        if ignore_c:
            ignore_c = False
        else:
            if c == '!':
                ignore_c = True
            elif c == '<':
                in_garbage = True   
            elif c == '>':
                in_garbage = False
            elif not in_garbage and c == '{':
                in_group = True
                group_depth += 1
                total_score += group_depth
            elif not in_garbage and c == '}':
                group_depth -= 1
    return total_score

In [53]:
assert(score_garbage_stream('{}') == 1)
assert(score_garbage_stream('{{{}}}') == 6)
assert(score_garbage_stream('{{},{}}') == 5)
assert(score_garbage_stream('{{{},{},{{}}}}') == 16)
assert(score_garbage_stream('{<a>,<a>,<a>,<a>}') == 1)
assert(score_garbage_stream('{{<ab>},{<ab>},{<ab>},{<ab>}}') == 9)
assert(score_garbage_stream('{{<!!>},{<!!>},{<!!>},{<!!>}}') == 9)
assert(score_garbage_stream('{{<a!>},{<a!>},{<a!>},{<ab>}}') == 3)

In [54]:
%%time
day_9_part1 = score_garbage_stream(input9)

CPU times: user 1.84 ms, sys: 0 ns, total: 1.84 ms
Wall time: 1.9 ms


In [55]:
print(day_9_part1)

9662


## [Day 9](http://adventofcode.com/2017/day/9): Stream Processing (part 2)

How many non-canceled characters are within the garbage in your puzzle input?

In [56]:
def count_garbage_chars(g_stream):
    in_group = False 
    in_garbage = False
    ignore_c = False
    
    garbage_chars = 0
    
    for c in g_stream:
        if ignore_c:
            ignore_c = False
        else:
            if c == '!':
                ignore_c = True
            elif c == '>':
                in_garbage = False                
            elif in_garbage:
                garbage_chars += 1
            elif c == '<':
                in_garbage = True
    return garbage_chars

In [57]:
assert(0 == count_garbage_chars('<>'))
assert(17 == count_garbage_chars('<random characters>'))
assert(3 == count_garbage_chars('<<<<>'))
assert(2 == count_garbage_chars('<{!>}>'))
assert(0 == count_garbage_chars('<!!>'))
assert(0 == count_garbage_chars('<!!!>>'))
assert(10 == count_garbage_chars('<{o"i!a,<{i<a>'))

In [58]:
%%time
day_9_part2 = count_garbage_chars(input9)

CPU times: user 1.07 ms, sys: 14 µs, total: 1.08 ms
Wall time: 1.09 ms


In [59]:
print(day_9_part2)

4903


## [Day 10](http://adventofcode.com/2017/day/10): Knot Hash (part 1)

However, you should instead use the standard list size of 256 (with values 0 to 255) and the sequence of lengths in your puzzle input. Once this process is complete, what is the result of multiplying the first two numbers in the list?

In [60]:
with open('./inputs/input_day_10.txt','r') as f:
    input10 = [x.strip('\n') for x in list(f)][0]
    input10 = [int(x) for x in input10.split(',')]

In [61]:
input10

[34, 88, 2, 222, 254, 93, 150, 0, 199, 255, 39, 32, 137, 136, 1, 167]

In [62]:
def circle_reverse(l, lo, hi):
    n = len(l)
    if hi < len(l):
        l[lo:hi+1] = list(reversed(l[lo:hi+1]))
    else:
        l_to_reverse = []
        for i in range(lo, hi + 1):
            l_to_reverse.append(l[i % n])
        l_to_reverse = list(reversed(l_to_reverse))
        for i in range(lo, hi + 1):
            l[i % n] =  l_to_reverse[(i - lo) % n]
    return l

def elf_hash(hash_list, input_lengths):
    n = len(hash_list)
    vals = list(range(n))
    skip_size = 0
    pos = 0
    for length in input_lengths:
        circle_reverse(vals, pos, pos + length - 1)
        pos = (pos + length + skip_size) % (n)
        skip_size += 1        
    return vals, pos, skip_size
        

In [63]:
eh = elf_hash([0,1,2,3,4], [3,4,1,5])
vals, pos, skip_size = eh
assert(vals[0] * vals[1] == 12)

In [64]:
eh = elf_hash(range(0,256), input10)
vals, pos, skip_size = eh
print(vals[0] * vals[1])

54675


## [Day 10](http://adventofcode.com/2017/day/10): Knot Hash (part 2)

Treating your puzzle input as a string of ASCII characters, what is the Knot Hash of your puzzle input? Ignore any leading or trailing whitespace you might encounter.

In [65]:
with open('./inputs/input_day_10.txt','r') as f: 
    input10v2 = [x.strip('\n') for x in list(f)][0]

In [66]:
from functools import reduce

In [67]:
def knot_hash(input_string):
    # convert the input character stream to their ascii values
    # and use them as lengths
    lengths = [ord(x) for x in list(input_string)]
    
    # append the extra lengths to the end
    extra_lengths = [17, 31, 73, 47, 23]
    lengths.extend(extra_lengths)

    # set up
    hash_list = list(range(256))
    pos = 0
    skip_size = 0
    
    # apply the knot algorithm to the hash_list 64 times
    for i in range(64):
        for length in lengths:
            circle_reverse(hash_list, pos, pos + length - 1)
            pos = (pos + length + skip_size) % (256)
            skip_size += 1

    # the result of 64 applications of the knot algorithm is the sparse hash
    sparse_hash = hash_list
    
    # compute the dense hash by XoR-ing each 16 number block of the sparse hash
    dense_hash = [0 for i in range(16)]
    for i in range(16):
        dense_hash[i] = reduce(lambda x,y : x^y, sparse_hash[i*16 : (i+1)*16])
        
    # extract the hexadecimal values from the dense hash
    dense_hash = ''.join([hex(x)[-2:] for x in dense_hash])
    dense_hash = dense_hash.replace('x', '0')
    
    return dense_hash

In [68]:
assert('a2582a3a0e66e6e86e3812dcb672a272' == knot_hash(''))
assert('33efeb34ea91902bb2f59c9920caa6cd' == knot_hash('AoC 2017'))
assert('3efbe78a8d82f29979031a4aa0b16a9d' == knot_hash('1,2,3'))
assert('63960835bcdc130f0b66d7ff4f6a5a8e' == knot_hash('1,2,4'))

In [69]:
knot_hash(input10v2)

'a7af2706aa9a09cf5d848c1e6605dd2a'

## [Day 11](http://adventofcode.com/2017/day/11): Hex Ed (part 1)


*warning* I'm pretty sure this is incomplete, and I didn't come up with a good solution. This gives the correct answer for the problem, but I don't trust it *at all* since it's not fully thought through on my part.

In [70]:
with open('./inputs/input_day_11.txt','r') as f: 
    input11 = f.readline().strip('\n')

In [71]:
import numpy as np

def reduce_directions(grid_path_str):
    grid_path = grid_path_str.split(',')
    
    cardinal_dirs = ['n','ne','nw','se','s','sw',]
    
    # extract all directions from the path
    dir_counts = dict()
    for d in cardinal_dirs:
        dir_counts[d] = grid_path.count(d)

    # reduce directions
    m = min(dir_counts['ne'], dir_counts['s'])
    dir_counts['ne'], dir_counts['s'], dir_counts['se'] = (dir_counts['ne'] - m, dir_counts['s'] - m, 
                                                           dir_counts['se'] + m)
    
    m = min(dir_counts['se'], dir_counts['sw'])
    dir_counts['se'], dir_counts['sw'], dir_counts['s'] = (dir_counts['se'] - m, dir_counts['sw'] - m, 
                                                           dir_counts['s'] +m)                                                           

    m = min(dir_counts['ne'], dir_counts['nw'])
    dir_counts['ne'], dir_counts['nw'], dir_counts['n'] = (dir_counts['ne'] - m, dir_counts['nw'] - m, 
                                                           dir_counts['n'] +m)

    m = min(dir_counts['ne'], dir_counts['sw'])
    dir_counts['ne'], dir_counts['sw'] = dir_counts['ne'] - m, dir_counts['sw'] - m

    m = min(dir_counts['se'], dir_counts['nw'])
    dir_counts['se'], dir_counts['nw'] = dir_counts['se'] - m, dir_counts['nw'] - m
        
    m = min(dir_counts['n'], dir_counts['s'])
    dir_counts['n'], dir_counts['s'] = dir_counts['n'] - m, dir_counts['s'] - m

    return dir_counts

def count_steps(grid_path_str):
    dir_counts = reduce_directions(grid_path_str)
    return sum(dir_counts.values())
#parse_dir('sw')

In [72]:
assert(count_steps('ne,ne,ne') == 3)
assert(count_steps('ne,ne,sw,sw') == 0)
assert(count_steps('ne,ne,s,s') == 2)
assert(count_steps('se,sw,se,sw,sw') == 3)
assert(count_steps('ne,nw,ne,nw,nw') == 3)

In [73]:
%%time
print(count_steps(input11))

1049
CPU times: user 2.07 ms, sys: 0 ns, total: 2.07 ms
Wall time: 1.95 ms


## [Day 11](http://adventofcode.com/2017/day/11): Hex Ed (part 2)

How many steps away is the furthest he ever got from his starting position?

What follows is a *VERY* bad solution. But it does the job, and I don't want to think about it right now. 

In [74]:
def max_steps_away(grid_path_str):
    grid_path = grid_path_str.split(',')
    l_path = len(grid_path)
    max_steps_away = 0
    for l in range(1, l_path):
        partial_path = grid_path[0:l]
        steps_taken = count_steps(','.join(partial_path))
        if steps_taken > max_steps_away:
            max_steps_away = steps_taken
    return max_steps_away

In [75]:
%%time
print(max_steps_away(input11))

2646
CPU times: user 5.67 s, sys: 272 ms, total: 5.94 s
Wall time: 5.95 s


## [Day 12](http://adventofcode.com/2017/day/12): Digital Plumber 

### Part 1

Walking along the memory banks of the stream, you find a small village that is experiencing a little confusion: some programs can't communicate with each other.

Programs in this village communicate using a fixed system of pipes. Messages are passed between programs using these pipes, but most programs aren't connected to each other directly. Instead, programs pass messages between each other until the message reaches the intended recipient.

For some reason, though, some of these messages aren't ever reaching their intended recipient, and the programs suspect that some pipes are missing. They would like you to investigate.

You walk through the village and record the ID of each program and the IDs with which it can communicate directly (your puzzle input). Each program has one or more programs with which it can communicate, and these pipes are bidirectional; if 8 says it can communicate with 11, then 11 will say it can communicate with 8.

You need to figure out how many programs are in the group that contains program ID 0.

For example, suppose you go door-to-door like a travelling salesman and record the following list:

    0 <-> 2
    1 <-> 1
    2 <-> 0, 3, 4
    3 <-> 2, 4
    4 <-> 2, 3, 6
    5 <-> 6
    6 <-> 4, 5

In this example, the following programs are in the group that contains program ID 0:

    Program 0 by definition.
    Program 2, directly connected to program 0.
    Program 3 via program 2.
    Program 4 via program 2.
    Program 5 via programs 6, then 4, then 2.
    Program 6 via programs 4, then 2.

Therefore, a total of 6 programs are in this group; all but program 1, which has a pipe that connects it to itself.

How many programs are in the group that contains program ID 0?

### Part Two 

There are more programs than just the ones in the group containing program ID 0. The rest of them have no way of reaching that group, and still might have no way of reaching each other.

A group is a collection of programs that can all communicate via pipes either directly or indirectly. The programs you identified just a moment ago are all part of the same group. Now, they would like you to determine the total number of groups.

In the example above, there were 2 groups: one consisting of programs 0,2,3,4,5,6, and the other consisting solely of program 1.

How many groups are there in total?

In [83]:
with open('./inputs/input_day_12.txt','r') as f:
    input12 = list(f)

In [84]:
import numpy as np

class UnionFindIntArray:
    def __init__(self, n_elts):
        self.n_elts = n_elts
        self.parent = list(range(n_elts))
        self.n_components = n_elts
        
    def find(self,x):
        if x != self.parent[x]:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x == root_y:
            pass
        else:
            self.parent[root_x] = root_y
            self.n_components -= 1
            
    def connected(self, x, y):
        return self.find(x) == self.find(y)

In [85]:
class ConnectedPrograms:
    def __init__(self, conxn_descr_list):
               
        # Program id we'll use to connect
        conxn_id = [x.strip('\n').split('<->')[0] for x in conxn_descr_list]
        conxn_id = [int(y.strip(' ')) for y in conxn_id]
        
        # Who is each program id connected to ?
        conxn_list = [x.strip('\n').split('<->')[1].split(',') for x in conxn_descr_list]
        conxn_list = [[int(y.strip(' ')) for y in x] for x in conxn_list]
        
        # Initialize union find
        self.n_elts = len(conxn_id)
        self.uf = UnionFindIntArray(self.n_elts)
        
        # connect them all up
        for i in range(self.n_elts):
            pgm = conxn_id[i]
            pgm_conxns = conxn_list[i]

            for connected_to_pgm in pgm_conxns:                
                self.uf.union(pgm, connected_to_pgm)                        
                
    def connected_to(self, x):
        is_connected_to_x = np.array([self.uf.connected(y,x) for y in range(self.n_elts)])
        connected_to_x = np.where(is_connected_to_x)[0]
        return list(connected_to_x)
    
    
    def n_connected_to(self, x):
        is_connected_to_x = self.connected_to(x)
        return len(is_connected_to_x)
    
    def n_components(self):
        return self.uf.n_components

In [86]:
descr = '''0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5'''

# union find structure from description
cp = ConnectedPrograms(descr.split('\n'))
assert(cp.n_connected_to(0) == 6)
assert(cp.n_components() == 2)

In [87]:
cp = ConnectedPrograms(input12)
print('Day 12 part 1: {}'.format(cp.n_connected_to(0)))
print('Day 12 part 1: {}'.format(cp.n_components()))


Day 12 part 1: 175
Day 12 part 1: 213


## [Day 13](): Packet Scanners (part 1)

Given the details of the firewall you've recorded, if you leave immediately, what is the severity of your whole trip?

In [88]:
with open('./inputs/input_day_13.txt','r') as f:
    input13 = list(f)

In [90]:
class FirewallTrip:
    def __init__(self, positions, heights, delay = 0):
        assert(len(positions) == len(heights))
        
        self.n = len(positions)
        self.positions = positions
        self.heights = heights
        
        self.delay = delay
        self.picosecond = 0
        self.layer = 0
        self.scanners = [0 for x in positions]
        self.scanner_dir = [1 for x in positions]
        self.severity = 0
        
    def update(self):        
        print (self.picosecond, self.layer, 
            [(self.positions[i], self.scanners[i]) for i in range(self.n)])
        if self.picosecond >= self.delay:
            # is the scanner there?
            if self.layer in self.positions:
                idx_layer = self.positions.index(self.layer)
                
                # update severity of trip
                if self.scanners[idx_layer] == 0:
                    dpth = self.positions[idx_layer]
                    rng  = self.heights[idx_layer]
                    sev  = dpth * rng
                    self.severity += sev
            
        # update positions of scanners
        for i in range(self.n):
            # change scanner direction
            if (self.scanners[i] == self.heights[i] - 1) or (self.scanners[i] == 0):
                if self.picosecond > 0:
                    self.scanner_dir[i] = -1 * self.scanner_dir[i]                
            self.scanners[i] += self.scanner_dir[i] 
            
        if self.picosecond >= self.delay:
            self.picosecond += 1
            self.layer += 1
        else:
            self.picosecond += 1
    
    def trip_severity(self, delay = 0):
        # reset just in case
        self.__init__(self.positions, self.heights, delay = delay)

        # run up until the last firewall
        #for i in range(max(self.positions) + 1):
        while self.layer < max(self.positions) + 1:
            self.update()
        return self.severity
    
    def find_best_delay(self):
        delay = 0        
        while True:            
            sev = self.trip_severity(delay)
            print (sev)
            if sev == 0:
                break
            else:
                delay += 1
        return delay

In [92]:
test_input_13 = (
'''0: 3
1: 2
4: 4
6: 4''')

#  put data in correct form
poshghts = [[int(y) for y in x.strip(' ').split(':')] for x in test_input_13.split('\n')]
pos = [x[0] for x in poshghts]
hghts = [x[1] for x in poshghts]

# build the trip simulator
ft = FirewallTrip(pos, hghts)

# run the test
assert(ft.trip_severity() == 24)

0 0 [(0, 0), (1, 0), (4, 0), (6, 0)]
1 1 [(0, 1), (1, 1), (4, 1), (6, 1)]
2 2 [(0, 2), (1, 0), (4, 2), (6, 2)]
3 3 [(0, 1), (1, 1), (4, 3), (6, 3)]
4 4 [(0, 0), (1, 0), (4, 2), (6, 2)]
5 5 [(0, 1), (1, 1), (4, 1), (6, 1)]
6 6 [(0, 2), (1, 0), (4, 0), (6, 0)]


In [95]:
poshghts = [[int(y) for y in x.strip(' ').split(':')] for x in input13]

In [96]:
poshghts = [[int(y) for y in x.strip(' ').split(':')] for x in input13]
pos = [x[0] for x in poshghts]
hghts = [x[1] for x in poshghts]

#ft = FirewallTrip(pos, hghts)
#print ft.trip_severity()

In [97]:
#ft.find_best_delay()
ft.trip_severity(10)

0 0 [(0, 0), (1, 0), (4, 0), (6, 0)]
1 0 [(0, 1), (1, 1), (4, 1), (6, 1)]
2 0 [(0, 2), (1, 0), (4, 2), (6, 2)]
3 0 [(0, 1), (1, 1), (4, 3), (6, 3)]
4 0 [(0, 0), (1, 0), (4, 2), (6, 2)]
5 0 [(0, 1), (1, 1), (4, 1), (6, 1)]
6 0 [(0, 2), (1, 0), (4, 0), (6, 0)]
7 0 [(0, 1), (1, 1), (4, 1), (6, 1)]
8 0 [(0, 0), (1, 0), (4, 2), (6, 2)]
9 0 [(0, 1), (1, 1), (4, 3), (6, 3)]
10 0 [(0, 2), (1, 0), (4, 2), (6, 2)]
11 1 [(0, 1), (1, 1), (4, 1), (6, 1)]
12 2 [(0, 0), (1, 0), (4, 0), (6, 0)]
13 3 [(0, 1), (1, 1), (4, 1), (6, 1)]
14 4 [(0, 2), (1, 0), (4, 2), (6, 2)]
15 5 [(0, 1), (1, 1), (4, 3), (6, 3)]
16 6 [(0, 0), (1, 0), (4, 2), (6, 2)]


0

In [98]:
ft.trip_severity(4)

0 0 [(0, 0), (1, 0), (4, 0), (6, 0)]
1 0 [(0, 1), (1, 1), (4, 1), (6, 1)]
2 0 [(0, 2), (1, 0), (4, 2), (6, 2)]
3 0 [(0, 1), (1, 1), (4, 3), (6, 3)]
4 0 [(0, 0), (1, 0), (4, 2), (6, 2)]
5 1 [(0, 1), (1, 1), (4, 1), (6, 1)]
6 2 [(0, 2), (1, 0), (4, 0), (6, 0)]
7 3 [(0, 1), (1, 1), (4, 1), (6, 1)]
8 4 [(0, 0), (1, 0), (4, 2), (6, 2)]
9 5 [(0, 1), (1, 1), (4, 3), (6, 3)]
10 6 [(0, 2), (1, 0), (4, 2), (6, 2)]


0